## Data

In [ ]:
from sklearn.datasets import load_wine
import pandas as pd

In [ ]:
data, target = load_wine(return_X_y=True)

In [ ]:
data_frame = pd.DataFrame(data=data, columns=load_wine().feature_names)

In [ ]:
data_frame.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


In [ ]:
from collections import Counter
Counter(target)

Counter({0: 59, 1: 71, 2: 48})

## Hyperparameter tuning

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV

### Decision Tree

In [ ]:
parameters = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 3, 4]
}
grid_search = GridSearchCV(param_grid=parameters, 
                           estimator=DecisionTreeClassifier())
grid_search.fit(data, target)
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

In [ ]:
best_parameters, best_score

({'criterion': 'gini',
  'min_samples_leaf': 2,
  'min_samples_split': 2,
  'splitter': 'random'},
 0.9331746031746032)

### Random forest

In [ ]:
parameters = {
    'n_estimators': [10, 25, 40, 50],
    'criterion': ['entropy', 'gini'],
    'min_samples_split': [2, 3, 4, 5]
}
grid_search = GridSearchCV(param_grid=parameters, 
                           estimator=RandomForestClassifier())
grid_search.fit(data, target)
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_
best_parameters, best_score

({'criterion': 'gini', 'min_samples_split': 2, 'n_estimators': 50},
 0.9888888888888889)

### KNN

In [ ]:
parameters = {
    'n_neighbors': [3, 5, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}
grid_search = GridSearchCV(param_grid=parameters, 
                           estimator=KNeighborsClassifier())
grid_search.fit(data, target)
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_
best_parameters, best_score

({'n_neighbors': 3, 'p': 1, 'weights': 'distance'}, 0.7814285714285714)

### SVM

In [ ]:
parameters = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [1, 1.5, 2],
    'degree': [2, 3, 4, 5]
}
grid_search = GridSearchCV(param_grid=parameters, 
                           estimator=SVC())
grid_search.fit(data, target)
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_
best_parameters, best_score

({'C': 1.5, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'},
 0.9666666666666668)

### Neural Network

In [ ]:
parameters = {
    'hidden_layer_sizes': [300, 400],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam', 'sgd', 'lbfgs'],
}
grid_search = GridSearchCV(param_grid=parameters, 
                           estimator=MLPClassifier())
grid_search.fit(data, target)
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_
best_parameters, best_score # {'activation': 'logistic', 'hidden_layer_sizes': 200, 'solver': 'adam'}

## Cross Validation


In [ ]:
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
tree_results = []
rf_results = []
knn_results = []
svm_results = []
mlp_results = []

In [ ]:
for i in range(30):
  kfold = KFold(n_splits=10, shuffle=True, random_state=i)
  tree = DecisionTreeClassifier(criterion='gini', min_samples_leaf=2,
                                min_samples_split=2, splitter='random')
  score = cross_val_score(tree, data, target, cv=kfold)
  tree_results.append(score.mean())

In [ ]:
for i in range(30):
  kfold = KFold(n_splits=10, shuffle=True, random_state=i)
  rf = RandomForestClassifier(criterion='gini', min_samples_split=2, 
                              n_estimators=50)
  score = cross_val_score(rf, data, target, cv=kfold)
  rf_results.append(score.mean())

In [ ]:
for i in range(30):
  kfold = KFold(n_splits=10, shuffle=True, random_state=i)
  knn = KNeighborsClassifier(n_neighbors=3, p=1, weights='distance')
  score = cross_val_score(knn, data, target, cv=kfold)
  knn_results.append(score.mean())

In [ ]:
for i in range(30):
  kfold = KFold(n_splits=10, shuffle=True, random_state=i)
  svm = SVC(C=1.5, gamma='scale', kernel='linear')
  score = cross_val_score(svm, data, target, cv=kfold)
  svm_results.append(score.mean())

In [ ]:
for i in range(30):
  kfold = KFold(n_splits=10, shuffle=True, random_state=i)
  mlp = MLPClassifier(activation='logistic', hidden_layer_sizes=200, solver='adam')
  score = cross_val_score(mlp, data, target, cv=kfold)
  mlp_results.append(score.mean())